<a href="https://colab.research.google.com/github/choycoy/choycoy.github.io/blob/main/ligthtgcn_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 62.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.


In [ ]:
!git clone https://github.com/microsoft/recommenders.git ./recommenders_microsoft

Cloning into './recommenders_microsoft'...
remote: Enumerating objects: 37376, done.
remote: Counting objects: 100% (908/908), done.
remote: Compressing objects: 100% (312/312), done.
remote: Total 37376 (delta 621), reused 759 (delta 580), pack-reused 36468
Receiving objects: 100% (37376/37376), 205.39 MiB | 29.10 MiB/s, done.
Resolving deltas: 100% (25279/25279), done.


In [ ]:
import sys
sys.path.append('/content/recommenders_microsoft')

In [ ]:
sys.path.append('/content/recommenders_microsoft/recommenders')

In [ ]:
import sys
import os
import scrapbook as sb
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]
Pandas version: 1.5.3
Tensorflow version: 2.11.0


In [ ]:
pip install retrying

In [ ]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m


# Model parameters


SEED = DEFAULT_SEED  # Set None for non-deterministic results

yaml_file = "/content/recommenders_microsoft/recommenders/models/deeprec/config/lightgcn.yaml"
user_file = "/content/recommenders_microsoft/recommenders/tests/resources/deeprec/lightgcn/user_embeddings.csv"
item_file = "/content/recommenders_microsoft/recommenders/tests/resources/deeprec/lightgcn/item_embeddings.csv"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
columns_name=['userID','rating', 'itemID']
df = pd.read_csv("/content/drive/My Drive/final/example_100000.csv",sep=",",names=columns_name)
print('User Dataset 1 shape : {}'.format(df.shape))

print(df)

User Dataset 1 shape : (100001, 3)
         userID rating   itemID
0       user_id  stars  item_id
1             1      3        1
2             2      5        2
3             3      3        3
4             4      5        4
...         ...    ...      ...
99996     79343      4     2791
99997     79344      1      785
99998     79345      4      662
99999     34711      2     1230
100000      397      5     5488

[100001 rows x 3 columns]


In [ ]:
df = df.drop(df.index[0])
# Assuming df is your DataFrame
df = df.reset_index(drop=True)

In [ ]:
df['rating'] = df['rating'].astype(float)


In [ ]:
df

,userID,rating,itemID
0,1,3.0,1
1,2,5.0,2
2,3,3.0,3
3,4,5.0,4
4,5,4.0,5
...,...,...,...
99995,79343,4.0,2791
99996,79344,1.0,785
99997,79345,4.0,662
99998,34711,2.0,1230


In [ ]:
train, test = python_stratified_split(df, ratio=0.75)

In [ ]:
data = ImplicitCF(train=train, test=test, seed=SEED)

In [ ]:
hparams = prepare_hparams(yaml_file,
                          n_layers=3,
                          batch_size= 1024,
                          epochs=100,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [ ]:
with Timer() as train_time:
    model.fit()
print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)1.7s: train loss = 0.40043 = (mf)0.40032 + (embed)0.00011
Epoch 2 (train)1.4s: train loss = 0.09520 = (mf)0.09483 + (embed)0.00037
Epoch 3 (train)1.4s: train loss = 0.04302 = (mf)0.04250 + (embed)0.00052
Epoch 4 (train)1.4s: train loss = 0.02465 = (mf)0.02404 + (embed)0.00061
Epoch 5 (train)1.4s + (eval)0.9s: train loss = 0.01585 = (mf)0.01517 + (embed)0.00068, recall = 0.05338, ndcg = 0.02931, precision = 0.00634, map = 0.02062
Epoch 6 (train)1.4s: train loss = 0.01131 = (mf)0.01058 + (embed)0.00074
Epoch 7 (train)1.5s: train loss = 0.00833 = (mf)0.00755 + (embed)0.00078
Epoch 8 (train)1.5s: train loss = 0.00647 = (mf)0.00566 + (embed)0.00082
Epoch 9 (train)1.4s: train loss = 0.00546 = (mf)0.00461 + (embed)0.00085
Epoch 10 (train)1.4s + (eval)0.9s: train loss = 0.00448 = (mf)0.00361 + (embed)0.00087, recall = 0.05295, ndcg = 0.02986, precision = 0.00621, map = 0.02157
Epoch 11 (train)1.4s: train loss = 0.00404 = (mf)0.00315 + (embed)0.00089
Epoch 12 (train)1.4s: train l

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head()

,userID,itemID,prediction
0,10002,1111,11.077831
1,10002,913,10.512508
2,10002,570,9.590033
3,10002,712,9.285477
4,10002,2371,9.224352


In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.017866
NDCG:	0.026090
Precision@K:	0.005898
Recall@K:	0.048439


In [ ]:
import pandas as pd

# Assuming you have a DataFrame called 'df' with columns 'userID', 'itemID', and 'prediction'

# Sort the DataFrame by 'userID'
sorted_df = topk_scores.sort_values(by='userID', ascending=True)

# Print the sorted DataFrame
print(sorted_df)


      userID itemID  prediction
0          1    657    5.875670
8          1   2114    4.064919
7          1   1254    4.087333
6          1   1081    4.326112
5          1     30    4.509099
...      ...    ...         ...
99982   9999    113    3.419796
99981   9999    657    3.502756
99980   9999    438    3.971323
99983   9999     81    3.379709
99988   9999   1382    2.908899

[793450 rows x 3 columns]


In [ ]:
import pandas as pd

# Assuming you have a DataFrame called 'topk_scores' with columns 'userID' and 'itemID'
# Group the DataFrame by 'userID' and aggregate the 'itemID' values as a list
grouped_df = topk_scores.groupby('userID')['itemID'].apply(list).reset_index()

# Create a new DataFrame with user ID and item ID list columns
result_df = pd.DataFrame({'userID': grouped_df['userID'], 'Item ID List': grouped_df['itemID']})

# Print the result DataFrame
print(result_df)


     userID                                       Item ID List
0     10002  [1111, 913, 570, 712, 2371, 1022, 40, 622, 345...
1     10005  [108, 469, 4313, 113, 3582, 246, 271, 4517, 44...
2     10017  [108, 45, 359, 1051, 463, 1491, 50, 1100, 329,...
3     10021  [886, 2874, 2276, 1081, 913, 623, 2440, 500, 2...
4     10027  [1114, 1913, 318, 1714, 1961, 319, 1262, 6989,...
...     ...                                                ...
3827   9957  [431, 1133, 4126, 7109, 813, 790, 108, 1980, 3...
3828   9959  [3659, 928, 592, 14, 2492, 443, 1961, 1603, 87...
3829   9969  [50, 592, 443, 1263, 1401, 1961, 108, 2186, 16...
3830    997  [684, 1678, 774, 1379, 816, 4928, 1437, 3178, ...
3831   9992  [697, 809, 2087, 21, 1337, 443, 126, 941, 529,...

[3832 rows x 2 columns]


In [ ]:
# Assuming you have a DataFrame called 'result_df' with columns 'userID' and 'Item ID List'

# Explode the 'Item ID List' column into separate rows
exploded_df = result_df.explode('Item ID List')

# Reset the index of the DataFrame
exploded_df = exploded_df.reset_index(drop=True)

# Print the concatenated list of item IDs
concatenated_list = exploded_df['Item ID List'].tolist()
print(concatenated_list)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:


def calculate_item_popularity(recommendation_list):
    item_counts = {}

    # Count the occurrences of each item in the recommendation list
    for item in recommendation_list:
        if item in item_counts:
            item_counts[item] += 1
        else:
            item_counts[item] = 1
    return item_counts


def calculate_entropy(popularity):
    # Normalize popularity values to probabilities
    total_count = sum(popularity.values())

    probabilities = [count / total_count for count in popularity.values()]
    # Calculate entropy
    entropy = -np.sum(probabilities * np.log2(probabilities))

    return entropy

from scipy import integrate

def calculate_gini_index(popularity):
    # Convert the dictionary values to a list
    popularity_values = list(popularity.values())

    # Calculate the CDF
    sorted_popularity = np.sort(popularity_values)[::-1]
    cdf = np.cumsum(sorted_popularity) / np.sum(sorted_popularity)

    # Calculate the Lorenz curve
    quantiles = np.linspace(0, 1, len(cdf))
    lorenz_curve = np.vstack((quantiles, cdf)).T

    # Calculate the area under the Lorenz curve
    area_A = np.sum((lorenz_curve[:-1, 0] + lorenz_curve[1:, 0]) * (lorenz_curve[1:, 1] - lorenz_curve[:-1, 1]) / 2)

    # Calculate the Gini Index
    gini_index = 0.5 - area_A

    return gini_index


# Calculate item popularity
item_popularity = calculate_item_popularity(concatenated_list)

gini_index = calculate_gini_index(item_popularity)
print("Gini Index:", gini_index)

# Calculate entropy
entropy = calculate_entropy(item_popularity)

print("Entropy:", entropy)


Gini Index: 0.4126729880628549
Entropy: 9.874286253174855


In [ ]:
unique_list = list(set(concatenated_list))

In [ ]:
unique_item_count = df['itemID'].nunique()


In [ ]:
coverage = len(unique_list) / unique_item_count
coverage

0.4621477990574551

In [ ]:
columns_name=['userID','rating', 'itemID', 'text']
df2 = pd.read_csv("/content/drive/My Drive/final/example_texts_10000.csv",sep=",",names=columns_name)
print('User Dataset 1 shape : {}'.format(df2.shape))
print(df2)

User Dataset 1 shape : (10001, 4)
        userID rating   itemID  \
0      user_id  stars  item_id   
1            1      3        1   
2            2      5        2   
3            3      3        3   
4            4      5        4   
...        ...    ...      ...   
9996      9468      5     1624   
9997      9469      5       73   
9998      9470      5      879   
9999      9471      5     1181   
10000     9472      2     3186   

                                                    text  
0                                                   text  
1      b"If you decide to eat here, just be aware it ...  
2      b"I've taken a lot of spin classes over the ye...  
3      b'Family diner. Had the buffet. Eclectic assor...  
4      b"Wow!  Yummy, different,  delicious.   Our fa...  
...                                                  ...  
9996   b'Excellent food and service. The place is fun...  
9997   b'Just about to get tucked into a meatloaf tha...  
9998   b'Outstanding custo

In [ ]:
df2 = df2.drop(df2.index[0])
# Assuming df is your DataFrame
df2['text'] = df2['text'].str[2:]
df2 = df2.reset_index(drop=True)

In [ ]:
df2

,userID,rating,itemID,text
0,1,3,1,"If you decide to eat here, just be aware it is..."
1,2,5,2,I've taken a lot of spin classes over the year...
2,3,3,3,Family diner. Had the buffet. Eclectic assortm...
3,4,5,4,"Wow! Yummy, different, delicious. Our favo..."
4,5,4,5,Cute interior and owner (?) gave us tour of up...
...,...,...,...,...
9995,9468,5,1624,Excellent food and service. The place is funct...
9996,9469,5,73,Just about to get tucked into a meatloaf that ...
9997,9470,5,879,Outstanding customer service! And my car is dr...
9998,9471,5,1181,I and my husband went here for Dinner one day ...


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

nltk.download('stopwords')
nltk.download('wordnet')

# Define the preprocessing function
def preprocess_text(text):
    # Tokenize the text
    tokens = text.lower().split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Remove punctuation
    tokens = [token.translate(str.maketrans('', '', string.punctuation)) for token in tokens]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df2['preprocessed_text'] = df2['text'].apply(preprocess_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from gensim import models, corpora, matutils
from sklearn.metrics.pairwise import cosine_similarity


# Step 2: Create a document-term matrix
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(df2['preprocessed_text'])

# Convert the document-term matrix to a compatible corpus
corpus = matutils.Sparse2Corpus(dtm.T)

# Step 3: Train the LDA model
num_topics = 45
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())
lda_model = models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=num_topics,
    passes=10,
    random_state=42
)

# Step 4: Infer topic distributions
document_topics = lda_model[corpus]

# Step 5: Find similar items for each item ID
similar_items_dict = {}
for i, doc_topics in enumerate(document_topics):
    item_id = df2['itemID'][i]
    if doc_topics:  # Check if the document has any inferred topics
        target_topic_dist = matutils.sparse2full(doc_topics, lda_model.num_topics)
        similar_items = []
        for j, doc_topics_j in enumerate(document_topics):
            if j != i and doc_topics_j:  # Exclude the target document and check if the document has any inferred topics
                doc_topic_dist = matutils.sparse2full(doc_topics_j, lda_model.num_topics)
                similarity_score = cosine_similarity([target_topic_dist], [doc_topic_dist])[0][0]
                similar_items.append((df2['itemID'][j], similarity_score))
        similar_items = sorted(similar_items, key=lambda x: x[1], reverse=True)[:5]
        similar_item_ids = [item[0] for item in similar_items]
        similar_items_dict[item_id] = similar_item_ids

# Create a DataFrame to store item IDs and similar item IDs
similar_items_df = pd.DataFrame(columns=['Item ID', 'Similar Item ID'])
for item_id, similar_items in similar_items_dict.items():
    similar_items_df = similar_items_df.append({'Item ID': item_id, 'Similar Item ID': similar_items}, ignore_index=True)

# Print the DataFrame
print(similar_items_df)

In [ ]:
# Step 4: Infer user-topic distributions
user_topic_distributions = lda_model[corpus]

# Step 5: Find similar users for each user ID
# Step 5: Find similar users for each user ID
similar_users_dict = {}
for i, user_topics in enumerate(user_topic_distributions):
    user_id = df2['userID'][i]
    if user_topics:  # Check if the user has any inferred topics
        target_topic_dist = matutils.sparse2full(user_topics, lda_model.num_topics)
        similar_users = []
        for j, user_topics_j in enumerate(user_topic_distributions):
            if j != i and user_topics_j:  # Exclude the target user and check if the user has any inferred topics
                user_topic_dist = matutils.sparse2full(user_topics_j, lda_model.num_topics)
                similarity_score = cosine_similarity([target_topic_dist], [user_topic_dist])[0][0]
                similar_users.append((df2['userID'][j], similarity_score))
        similar_users = sorted(similar_users, key=lambda x: x[1], reverse=True)[:5]
        similar_user_ids = [user[0] for user in similar_users]
        similar_user_ids = list(set(similar_user_ids))  # Remove duplicates
        similar_users_dict[user_id] = similar_user_ids

# Create a DataFrame to store user IDs and similar user IDs
similar_users_df = pd.DataFrame(columns=['User ID', 'Similar User ID'])

# Populate the DataFrame with user IDs and similar user IDs
for user_id, similar_users in similar_users_dict.items():
    similar_users_df = similar_users_df.append({'User ID': user_id, 'Similar User ID': similar_users}, ignore_index=True)

# Print the DataFrame
print(similar_users_df)

In [ ]:
import pandas as pd

# Assuming similar_items_df and df are your DataFrames
# matching based on 'Item ID' and 'itemID' columns respectively

# Merge the DataFrames based on the matching 'Item ID' and 'itemID'
merged_df = pd.merge(df, similar_items_df, left_on='itemID', right_on='Item ID', how='left')

# Assign the 'Similar Item ID' values to the new column 'similar_item_id'
df['similar_item_id'] = merged_df['Similar Item ID']

# Drop unnecessary columns if needed
merged_df.drop(['Item ID', 'Similar Item ID'], axis=1, inplace=True)

# Print the updated DataFrame
print(merged_df)


In [ ]:
import pandas as pd

# Assuming similar_items_df and df are your DataFrames
# matching based on 'Item ID' and 'itemID' columns respectively

# Merge the DataFrames based on the matching 'Item ID' and 'itemID'
merged_df2 = pd.merge(df, similar_users_df, left_on='User ID', right_on='userID', how='left')

# Assign the 'Similar Item ID' values to the new column 'similar_item_id'
merged_df['similar_item_id'] = merged_df2['Similar Item ID']

# Drop unnecessary columns if needed
merged_df.drop(['Item ID', 'Similar Item ID'], axis=1, inplace=True)

# Print the updated DataFrame
print(merged_df)
